In [2]:
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.memory import VectorStoreRetrieverMemory
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from datetime import datetime, timezone
from langchain.memory import ConversationBufferMemory, VectorStoreRetrieverMemory, CombinedMemory
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
llm = HuggingFaceEndpoint(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    temperature=0.3
)
model = ChatHuggingFace(llm=llm)

d:\anaconda3\envs\genai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
# Memory setup
buffer_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

memory_store = FAISS.from_texts([""], embeddings)
memory_retriever = memory_store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 10, 'lambda_mult': 0.7}
)
vector_memory = VectorStoreRetrieverMemory(retriever=memory_retriever)

memory = CombinedMemory(memories=[buffer_memory, vector_memory])

C:\Users\pc\AppData\Local\Temp\ipykernel_6524\365941363.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory(
C:\Users\pc\AppData\Local\Temp\ipykernel_6524\365941363.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  vector_memory = VectorStoreRetrieverMemory(retriever=memory_retriever)
C:\Users\pc\AppData\Local\Temp\ipykernel_6524\365941363.py:14: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on chat_memory=InMemoryChatMessageHistory(messages=[]) return_messages=True memory_key='chat_history'
  memory = CombinedMemory(memories=[buffer_memory, vector_memory])


In [7]:
prompt_template = """
You are Krishi Mitra, an advanced AI agricultural assistant specialized in helping farmers.

Return responses in clean, structured HTML format with proper formatting:
- Use <h3>, <h4> for headings
- Use <p> for paragraphs
- Use <ul>, <li> for lists
- Use <strong> for emphasis
- Use <br> for line breaks when needed

Current Weather Information:
{weather_context}

Conversation History (use if relevant):
{history}

User Query: {query}

Provide practical, actionable advice based on the weather conditions and farming best practices.
Your response:
"""

In [8]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["weather_context", "history", "query"]
)

In [9]:
# def format_docs(retrieved_docs):
#     return "\n\n".join(
#         f"[Chunk {i}] {doc.page_content}" for i, doc in enumerate(retrieved_docs, 1)
#     )

In [10]:
parser = StrOutputParser()

In [11]:
def get_history(query: str):
    history_vars = memory.load_memory_variables({'input': query})
    history = history_vars.get("chat_history", [])
    if isinstance(history, list):
        formatted_history = "\n".join(
            f"{msg.type.upper()}: {msg.content}" for msg in history
        )
    else:
        formatted_history = history
    return formatted_history


In [12]:
def get_weather_context(query: str):
    weather_info = """
    <h4>Current Weather:</h4>
    <p>Location: Delhi, India</p>
    <p>Temperature: 28°C</p>
    <p>Humidity: 70%</p>
    <p>Conditions: Partly Cloudy</p>
    """
    return weather_info

In [13]:
parallel_chain = RunnableParallel({
    'weather_context': RunnableLambda(get_weather_context),
    'history': RunnableLambda(get_history),
    'query': RunnablePassthrough()
})

In [14]:
main_chain = parallel_chain | prompt | model | parser

In [15]:
def ask(query: str):
    answer = main_chain.invoke(query)
    buffer_memory.save_context({"input": query}, {"output": answer})
    vector_memory.save_context(
        {"input": query},
        {"output": answer, "timestamp": datetime.now(timezone.utc).isoformat()}
    )

    return answer

In [16]:
def reset_history():
    buffer_memory.clear()

    memory_store = FAISS.from_texts([""], embeddings)
    memory_retriever = memory_store.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 5, 'fetch_k': 10, 'lambda_mult': 0.7}
    )
    vector_memory = VectorStoreRetrieverMemory(retriever=memory_retriever)

    memory.memories = [buffer_memory, vector_memory]

In [17]:
# reset_history()

In [18]:
ask("How can i improve crop yield given the current weather conditions?")

ValueError: You must provide an api_key to work with novita API or log in with `hf auth login`.

In [ ]:
ask("How can i cure my potato plants from blight disease?")

'The reviews suggest that the price of the laptop is reasonable, with one buyer mentioning they got it for ₹47,000 after a discount, and another saying it\'s "worth every penny" at the price of ₹52,999, which is already 43% off from the original price of ₹93,690.'

In [ ]:
ask("What's the best time to plant this crop?")

'The most negative review about this product states that it\'s "only good for web browsing and watching videos" and that the sound quality is "not good". The reviewer mentions that their phone has louder sound than this laptop, and it\'s not smooth for Excel work, making it not suitable for office workers who do a lot of Excel work. The reviewer suggests that students or those preparing for job exams might find it suitable, but recommends considering a higher variant for better performance.'